In [4]:
import mysql.connector
import pandas as pd 
import numpy as np 
from config import HOST, DATABASE, USERNAME, PASSWORD

In [7]:
# connecting to grocery db

try:
    connection = mysql.connector.connect(
        host=host,
        database=database,
        user=user,
        password=password
    )

    if connection.is_connected():
        print("Connected to MySQL database")

        # Your database operations go here

except mysql.connector.Error as err:
    print(f"Error: {err}")

# finally:
    # Close the connection in the finally block to ensure it's always closed
    #if 'connection' in locals() and connection.is_connected():
    #    connection.close()
    #    print("Connection closed")

Connected to MySQL database


yum yum


In [ ]:
f